# Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 


Just make sure:

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2. to generate maps to visualize your neighborhoods and how they cluster together.


Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. **(3 marks)**

Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


### Repeating steps from Question 1 and Question 2

In [2]:
df = pd.read_html ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames , inplace=True)
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.sort_values('Postal Code', inplace=True)
df.reset_index(drop=True, inplace=True)

Reading the Toronto Neighbourhood coordinates csv file into dataframe

In [3]:
df_coord = pd.read_csv ('https://cocl.us/Geospatial_data')

Merging the two dataframes

In [4]:
df_new = pd.merge(df, df_coord, on='Postal Code', how='inner')
df_new

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_new['Borough'].unique()),
        df_new.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [6]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### Using geopy library to get the latitude and longitude values of Toronto City.

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Creating a map of New York with neighborhoods superimposed on top.

In [8]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

###  Using just the location of Downtown Toronto for analysis in this Question

In [9]:
downtown_data = df_new[df_new['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
print(downtown_data.shape)
downtown_data.head()

(19, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


geographical coordinates of Downtown Toronto

In [10]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [11]:
# create map of Downtown Toronto using latitude and longitude values
map_dtor = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighbourhood in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighbourhood']):
    label = folium.Popup(neighbourhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtor)  
    
map_dtor

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Exploring Neighborhoods

In [13]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        with open('results.json', 'w') as outfile:
            json.dump(results, outfile, indent=4)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
downtown_venues= getNearbyVenues(names=downtown_data['Neighbourhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [16]:
print(downtown_venues.shape)
downtown_venues.head()

(1232, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Grouping by Neighbourhood

In [17]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


#### unique categories from all the returned venues

In [18]:
# one hot encoding
dt_cat = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
dt_cat['Neighbourhood'] = (downtown_venues['Neighbourhood'])

dt_cat.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Rosedale
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Rosedale
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Rosedale
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Rosedale
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"St. James Town, Cabbagetown"


In [19]:
# move neighbourhood column to the first column
fixed_columns = [dt_cat.columns[-1]] + list(dt_cat.columns[:-1])
dt_cat = dt_cat[fixed_columns]
dt_cat.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dt_cat.shape

(1232, 212)

#### Grouping rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [21]:
downtown_grouped = dt_cat.groupby('Neighbourhood').mean().reset_index()
downtown_grouped.shape

(19, 212)

#### displaying the top 5 venues for each neighborhood.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Pharmacy,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Breakfast Spot
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Boat or Ferry,Sculpture Garden,Rental Car Location,Bar,Plane,Airport Gate
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Burger Joint,Thai Restaurant,Bubble Tea Shop,Poke Place,Department Store
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Baby Store,Nightclub,Coffee Shop,Restaurant,Candy Store,Italian Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Café,Fast Food Restaurant,Hotel,Pub


## Cluster Neighborhoods

Using **_k_-means** to cluster the neighborhood into 3 clusters.

In [24]:
# set number of clusters
kclusters = 3

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)

new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

downtown_merged = downtown_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Trail,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Pizza Place,Coffee Shop,Café,Bakery,Restaurant,Pub,Italian Restaurant,Caribbean Restaurant,Outdoor Sculpture,Sandwich Place
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Café,Fast Food Restaurant,Hotel,Pub
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Café,Park,Pub,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Italian Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Diner,Italian Restaurant,Hotel


Creating a cluster Map

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

#### Cluster 1

In [27]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Park,Trail,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 2

In [28]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Pizza Place,Coffee Shop,Café,Bakery,Restaurant,Pub,Italian Restaurant,Caribbean Restaurant,Outdoor Sculpture,Sandwich Place
2,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Café,Fast Food Restaurant,Hotel,Pub
3,Downtown Toronto,1,Coffee Shop,Café,Park,Pub,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Brewery,Italian Restaurant
4,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Bubble Tea Shop,Diner,Italian Restaurant,Hotel
5,Downtown Toronto,1,Coffee Shop,Café,Clothing Store,Cocktail Bar,Cosmetics Shop,American Restaurant,Creperie,Moroccan Restaurant,Cheese Shop,Park
6,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Pharmacy,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Breakfast Spot
7,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Burger Joint,Thai Restaurant,Bubble Tea Shop,Poke Place,Department Store
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Gym,Thai Restaurant,Deli / Bodega,Clothing Store,Steakhouse,Bookstore
9,Downtown Toronto,1,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Scenic Lookout,Brewery,Fried Chicken Joint,Restaurant,Bar
10,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Salad Place,American Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop


#### Cluster 3

In [29]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Boat or Ferry,Sculpture Garden,Rental Car Location,Bar,Plane,Airport Gate
